# Data Science - Module 3 - Final Project Submission

* Student Name: **James Toop**
* Student Pace: **Self Paced**
* Scheduled project review date/time: **TBC**
* Instructor name: **Jeff Herman**
* Blog post URL: **TBC**

## Table of Contents

---
<a name="business-case"></a>
## 1. Business Case / Project Scope

Tanzania has a water and sanitation crisis. Only 50% of the population of 53 million have access to an improved source of safe water, and 34% of the population has access to improved sanitation. The demand for both water and sanitation is high.

Water is an essential of life, yet millions around the world still don’t have access to clean water. One of the most common causes of death in the developing world is drinking dirty and diseased water.

Did you know 748 Million people in the world don’t have access to safe water?

Water wells provide clean water for years. In rural areas, they are a lifeline for the inhabitants as this may be the only source of potable water.

Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, 
which need some repairs, and which don't work at all? This is an intermediate-level practice competition. 
Predict one of these three classes based on a number of variables about what kind of pump is operating, 
when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve 
maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

The goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

---
<a name="eda"></a>
## 2. Exploratory Data Analysis (EDA)

<a name="data-discovery"></a>
### 2A. Data Discovery

This section presents an initial step to investigate, understand and document the available data fields and relationships, highlighting any potential issues / shortcomings within the datasets supplied.

In [2]:
# Import the relevant libraries for data discovery and exploratory data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Set styles and color palette for Seaborn
plt.style.use('seaborn-whitegrid')
pres_palette = ['#599191','#8dd8d3','#0b6374','#c0791b','#424242','#fd5b58','#d7e6a3','#d558ab','#27278b']
sns.set_palette(sns.color_palette(pres_palette))

In [3]:
# Import the waterpoints training data file from the repository then inspect the data
waterpoints = pd.read_csv('training-set-values.csv')
waterpoints.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
waterpoints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [5]:
waterpoints.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [84]:
# View unique values for category fields (Refactored to streamline repetitive code into a function)

def output_field_values(fields):
    for field in fields:        
        field_values = np.sort(waterpoints[field].astype(str).unique())
        print(len(field_values),"\033[1m",field,"\033[0m options:\n")
        for field_value in field_values:
            print("\t",field_value)
        print("\n")


output_field_values(['scheme_management',
                     'permit',
                     'extraction_type',
                     'extraction_type_group',
                     'extraction_type_class',
                     'management',
                     'management_group',
                     'payment',
                     'payment_type',
                     'water_quality',
                     'quality_group',
                     'quantity',
                     'quantity_group',
                     'source',
                     'source_type',
                     'source_class',
                     'waterpoint_type',
                     'waterpoint_type_group'
                    ])

13  scheme_management  options:

	 Company
	 None
	 Other
	 Parastatal
	 Private operator
	 SWC
	 Trust
	 VWC
	 WUA
	 WUG
	 Water Board
	 Water authority
	 nan


3  permit  options:

	 False
	 True
	 nan


18  extraction_type  options:

	 afridev
	 cemo
	 climax
	 gravity
	 india mark ii
	 india mark iii
	 ksb
	 mono
	 nira/tanira
	 other
	 other - mkulima/shinyanga
	 other - play pump
	 other - rope pump
	 other - swn 81
	 submersible
	 swn 80
	 walimi
	 windmill


13  extraction_type_group  options:

	 afridev
	 gravity
	 india mark ii
	 india mark iii
	 mono
	 nira/tanira
	 other
	 other handpump
	 other motorpump
	 rope pump
	 submersible
	 swn 80
	 wind-powered


7  extraction_type_class  options:

	 gravity
	 handpump
	 motorpump
	 other
	 rope pump
	 submersible
	 wind-powered


12  management  options:

	 company
	 other
	 other - school
	 parastatal
	 private operator
	 trust
	 unknown
	 vwc
	 water authority
	 water board
	 wua
	 wug


5  management_group  options:

	 commerc

In [86]:
waterpoints['construction_year'].value_counts().sort_index()

0       20709
1960      102
1961       21
1962       30
1963       85
1964       40
1965       19
1966       17
1967       88
1968       77
1969       59
1970      411
1971      145
1972      708
1973      184
1974      676
1975      437
1976      414
1977      202
1978     1037
1979      192
1980      811
1981      238
1982      744
1983      488
1984      779
1985      945
1986      434
1987      302
1988      521
1989      316
1990      954
1991      324
1992      640
1993      608
1994      738
1995     1014
1996      811
1997      644
1998      966
1999      979
2000     2091
2001      540
2002     1075
2003     1286
2004     1123
2005     1011
2006     1471
2007     1587
2008     2613
2009     2533
2010     2645
2011     1256
2012     1084
2013      176
Name: construction_year, dtype: int64

In [88]:
waterpoints['installer'].value_counts().sort_index()

-                   3
0                 777
A.D.B               1
AAR                 4
ABASIA             29
                 ... 
water board         2
wizara ya maji      2
world               4
world banks         1
world vision        1
Name: installer, Length: 2145, dtype: int64

In [12]:
# Check to see how many NULL values are contained within the dataset and in which fields
waterpoints.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [11]:
# Import the waterpoints training labels data file from the repository then inspect the data
waterpoints_status = pd.read_csv('training-set-labels.csv')
waterpoints_status['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

<a name="updated-field-descriptions"></a>
### 2B. Updated Field Descriptions


* **`amount_tsh`** : Total static head (amount of water available to waterpoint)
* **`date_recorded`** : The date the row was entered
* **`funder`** : Who funded the well
* **`gps_height`** : Altitude of the well
* **`installer`** : Organization that installed the well
* **`longitude`** : GPS coordinate
* **`latitude`** : GPS coordinate
* **`wpt_name`** : Name of the waterpoint if there is one
* **`num_private`** : NO FIELD DEFINITION CONSIDER DROPPING
* **`basin`** : Geographic water basin
* **`subvillage`** : Geographic location
* **`region`** : Geographic location
* **`region_code`** : Geographic location (coded)
* **`district_code`** : Geographic location (coded)
* **`lga`** : Geographic location
* **`ward`** : Geographic location
* **`population`** : Population around the well
* **`public_meeting`** : NO FIELD DEFINITION CONSIDER DROPPING
    * True
    * False
* **`recorded_by`** : Group entering this row of data
* **`scheme_management`** : Who operates the waterpoint
    * VWC
    * WUG
    * Water authority
    * WUA
    * Water Board
    * Parastatal
    * Private operator
    * Company
    * Other
    * SWC
    * Trust
    * None
* **`scheme_name`** : Who operates the waterpoint : LARGE NUMBER OF NULL VALUES / RELEVANCE CONSIDER DROPPING?
* **`permit`** : If the waterpoint is permitted
    * True
    * False
* **`construction_year`** : Year the waterpoint was constructed
* **`extraction_type`** : The kind of extraction the waterpoint uses (brand names of pumps)
    * gravity
    * nira/tanira
    * other
    * submersible
    * swn 80
    * mono
    * india mark ii
    * afridev
    * ksb 
    * other - rope pump
    * other - swn 81
    * windmill
    * india mark iii
    * cemo
    * other - play pump
    * walimi
    * climax
    * other - mkulima/shinyanga
* **`extraction_type_group`** : The kind of extraction the waterpoint uses : DUPLICATED DETAIL, CONSIDER DROPPING?
    * gravity
    * nira/tanira
    * other
    * submersible
    * swn 80
    * mono
    * india mark ii
    * afridev
    * rope pump
    * other handpump
    * other motorpump
    * wind-powered
    * india mark iii
* **`extraction_type_class`** : The kind of extraction the waterpoint uses : DUPLICATED DETAIL, CONSIDER DROPPING?
    * gravity
    * handpump
    * other
    * submersible
    * motorpump
    * rope pump
    * wind-powered
* **`management`** : How the waterpoint is managed
    * vwc
    * wug
    * water board
    * wua
    * private operator
    * [parastatal](https://www.collinsdictionary.com/dictionary/english/parastatal)
    * water authority
    * other
    * company
    * unknown
    * other - school
    * trust
* **`management_group`** : How the waterpoint is managed : DUPLICATED DETAIL, CONSIDER DROPPING?
    * user-group
    * commercial
    * [parastatal](https://www.collinsdictionary.com/dictionary/english/parastatal)
    * other
    * unknown
* **`payment`** : Whether the water is paid for and how : DUPLICATED DETAIL, CONSIDER DROPPING?
    * never pay
    * pay per bucket
    * pay monthly
    * pay when scheme fails
    * pay annually
    * other
    * unknown
* **`payment_type`** : Whether the water is paid for and how
    * never pay
    * per bucket
    * monthly
    * on failure
    * annually
    * other
    * unknown
* **`water_quality`** : The quality of the water
    * soft
    * salty
    * soft
    * salty
    * milky
    * coloured
    * salty abandoned
    * fluoride
    * flouride abandoned
    * unknown
* **`quality_group`** : The quality of the water : DUPLICATED DETAIL, CONSIDER DROPPING?
    * good
    * salty
    * milky
    * colored
    * fluoride
    * unknown
* **`quantity`** : The quantity of water :
    * enough
    * insufficient
    * dry
    * seasonal
    * unknown
* **` quantity_group`** : The quantity of water : DUPLICATED DETAIL, CONSIDER DROPPING?
    * enough
    * insufficient
    * dry
    * seasonal
    * unknown
* **`source`** : The source of the water
    * spring
    * shallow well
    * machine dbh
    * river
    * rainwater harvesting
    * hand dtw
    * lake
    * dam
    * other
    * unknown
* **`source_type`** : The source of the water : DUPLICATED DETAIL, CONSIDER DROPPING?
    * spring
    * shallow well
    * borehole
    * river/lake
    * rainwater harvesting
    * dam
    * other
* **`source_class`** : The source of the water
    * groundwater
    * surface
    * unknown
* **`waterpoint_type`** : The kind of waterpoint :
    * communal standpipe
    * hand pump
    * communal standpipe multiple
    * improved spring
    * cattle trough
    * dam
    * other    
* **`waterpoint_type_group`** : The kind of waterpoint : DUPLICATED DETAIL, CONSIDER DROPPING?
    * communal standpipe
    * hand pump
    * improved spring
    * cattle trough
    * dam
    * other

<a name="data-preprocessing"></a>
### 2C. Data Preprocessing

In [6]:
# Check for duplicate records
len(waterpoints[waterpoints.duplicated()])

0

In [90]:
# Drop specific fields containing duplicated, unnecessary or significantly incomplete data
waterpoints_processed = waterpoints.drop(['date_recorded',
                                          'funder',
                                          'wpt_name',
                                          'num_private',
                                          'public_meeting',
                                          'recorded_by',
                                          'scheme_name',
                                          'extraction_type_group',
                                          'extraction_type_class',
                                          'management_group',
                                          'payment',
                                          'quality_group',
                                          'quantity_group',
                                          'source_type',
                                          'waterpoint_type_group'
                                         ], axis=1)
waterpoints_processed.head()

,id,amount_tsh,gps_height,installer,longitude,latitude,basin,subvillage,region,region_code,...,permit,construction_year,extraction_type,management,payment_type,water_quality,quantity,source,source_class,waterpoint_type
0,69572,6000.0,1390,Roman,34.938093,-9.856322,Lake Nyasa,Mnyusi B,Iringa,11,...,False,1999,gravity,vwc,annually,soft,enough,spring,groundwater,communal standpipe
1,8776,0.0,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Nyamara,Mara,20,...,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe
2,34310,25.0,686,World vision,37.460664,-3.821329,Pangani,Majengo,Manyara,21,...,True,2009,gravity,vwc,per bucket,soft,enough,dam,surface,communal standpipe multiple
3,67743,0.0,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,...,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple
4,19728,0.0,0,Artisan,31.130847,-1.825359,Lake Victoria,Kyanyamisa,Kagera,18,...,True,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe


In [ ]:
# Create a new "age" field and replace all missing values with the median value